<a href="https://colab.research.google.com/github/pareshrchaudhary/experiments/blob/main/fundamentals/linear_regression_object_oriented.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Mathematical Explanation of Linear Regression

Assumption - The relationship between features
and target is approximately linear, i.e. expected value of the target can be expressed as a weighted sum of the features. This means that the change in the target variable is proportional to the change in the features. Many real-world relationships can be approximated well by a linear model.

Let's consider a simple linear regression model. Given a dataset $\{(x^{(1)}, y^{(1)}), (x^{(2)}, y^{(2)}), \ldots, (x^{(m)}, y^{(m)})\}$ where $x^{(i)}$ represents the feature and $y^{(i)}$ represents the target for the $i$th example, we want to learn a linear relationship between the features and the target.

The linear regression model assumes the following relationship between the features and the target:

\begin{equation}
y^{(i)} = b + \mathbf{w}^T \mathbf{x}^{(i)}
\end{equation}

where $b$ is the bias term (intercept), $\mathbf{w}$ is the weight vector (coefficients), and $\mathbf{x}^{(i)}$ is the feature vector of the $i$th example.

Our goal is to learn the parameters $b$ and $\mathbf{w}$ that minimize the mean squared error (MSE) between the predicted values and the actual values:

\begin{equation}
\text{MSE} = \frac{1}{m} \sum_{i=1}^{m} (y^{(i)} - (\mathbf{w}^T \mathbf{x}^{(i)} + b))^2
\end{equation}

To minimize the MSE, we use the gradient descent algorithm. In each iteration of gradient descent, we update the parameters $b$ and $\mathbf{w}$ according to the following update rules:

\begin{align}
b &:= b - \alpha \frac{1}{m} \sum_{i=1}^{m} (p_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \\
\mathbf{w} &:= \mathbf{w} - \alpha \frac{1}{m} \sum_{i=1}^{m} (p_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \mathbf{x}^{(i)}
\end{align}

where $p_{\mathbf{w},b}(\mathbf{x})$ is the predicted value given by the linear regression model and $\alpha$ is the learning rate.

The above update rules are implemented in the linear regression class using NumPy.


# Setup

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
!wget https://raw.githubusercontent.com/pareshrchaudhary/experiments/main/fundamentals/data/home_data.csv

--2024-02-07 16:58:50--  https://raw.githubusercontent.com/pareshrchaudhary/experiments/main/fundamentals/data/home_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2381890 (2.3M) [text/plain]
Saving to: ‘home_data.csv’

home_data.csv       100%[===================>]   2.27M  --.-KB/s    in 0.08s   

2024-02-07 16:58:51 (28.8 MB/s) - ‘home_data.csv’ saved [2381890/2381890]



# Train/Test data split

In [9]:
data = pd.read_csv('home_data.csv')
data.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161300e+04,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,4.580302e+09,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652
std,2.876566e+09,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631
min,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [10]:
# Convert the DataFrame to a NumPy array
data_array = data.to_numpy()

# Shuffle the data
np.random.shuffle(data_array)

# Define the proportion of data to be used for training
train_ratio = 0.8
num_train = int(train_ratio * len(data_array))

# Split the data into training and test sets
train_data = data_array[:num_train]
test_data = data_array[num_train:]

# Split the features and target variable for training set
X_train = np.float64(train_data[:, 3:])
# X_train = (X_train - np.min(X_train, axis=0)) / (np.max(X_train, axis=0) - np.min(X_train, axis=0))
y_train = np.float64(train_data[:, 2])

# Split the features and target variable for test set
X_test = np.float64(test_data[:, 3:])
# X_test = (X_test - np.min(X_test, axis=0)) / (np.max(X_test, axis=0) - np.min(X_test, axis=0))
y_test = np.float64(test_data[:, 2])

print(X_train.shape, y_train.shape, X_test.shape ,y_test.shape)

(17290, 18) (17290,) (4323, 18) (4323,)


# Model

In [5]:
class LinearRegression:
  def __init__(self):
    self.weights = None
    self.bias = None

  def initialize_parameters(self, input_dim):
    self.weights = np.zeros(input_dim)
    self.bias = 0

  def fit(self, X, y, optimizer):
    # Initialize model parameters
    self.initialize_parameters(X.shape[1])

    # Optimize parameters
    optimizer.optimize(self, X, y)

  def predict(self, X):

    # Make predictions
    return np.matmul(X, self.weights) + self.bias

class GradientDescentOptimizer:
  def __init__(self, learning_rate=0.01, iterations=1000, clip_value=1.0):
    self.learning_rate = learning_rate
    self.iterations = iterations
    self.clip_value = clip_value

  def optimize(self, model, X, y):
    for _ in range(self.iterations):
        predictions = model.predict(X)
        errors = ((predictions - y) ** 2 )/ 2*len(predictions)
        gradient_w = np.dot(X.T, -errors) / len(X)
        gradient_b = np.sum(-errors) / len(X)

        # Gradient clipping
        total_gradient_norm = np.sum(gradient_w**2) + gradient_b**2
        if total_gradient_norm > self.clip_value**2:
            multiplier = self.clip_value / np.sqrt(total_gradient_norm)
            gradient_w *= multiplier
            gradient_b *= multiplier

        # Update weights and bias
        model.weights -= self.learning_rate * gradient_w
        model.bias -= self.learning_rate * gradient_b

# Training

In [6]:
# Instantiate the LinearRegression class
model = LinearRegression()

# Instantiate the GradientDescentOptimizer class
optimizer = GradientDescentOptimizer(learning_rate=0.001, iterations=100000, clip_value=1.0)

# Fit the model
model.fit(X_train, y_train, optimizer)

In [7]:
from sklearn.metrics import mean_squared_error

np.sqrt(mean_squared_error(y_test, model.predict(X_test)))

8413380.422980024